In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
credit = pd.read_csv("../homecredit_data/home_feature_all.csv")

In [8]:
X_train = credit[credit['TARGET'].notnull()].drop(['TARGET'],axis=1)
X_test = credit[credit['TARGET'].isnull()]
y_train = credit['TARGET']

In [9]:
from sklearn.model_selection import KFold

ntrain = X_train.shape[0]
ntest = X_test.shape[0]
SEED = 0 #for reproducibility
NFOLDS = 3 # set folds for out-of-fold prediction
kf = KFold(n_splits = NFOLDS,random_state=SEED,shuffle=False)

def get_out_fold(clf,x_train,y_train,x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS,ntest))
    
    for i, (train_index,test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]
        
        clf.fit(x_tr,y_tr)
        
        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i,:] = clf.predict(x_test)
        
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1,1),oof_test.reshape(-1,1)

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
 
rf = RandomForestClassifier(n_estimators=500,warm_start=True,max_features='sqrt',max_depth=6,min_samples_split=3,min_samples_leaf=2,n_jobs=-1,verbose=0)
ada = AdaBoostClassifier(n_estimators=500,learning_rate=0.1)
et = ExtraTreesClassifier(n_estimators=500,n_jobs=-1,max_depth=8,min_samples_leaf=2,verbose=0)
gb = GradientBoostingClassifier(n_estimators=500,learning_rate=0.008,min_samples_split=3,min_samples_leaf=2,max_depth=5,verbose=0)

#dt = DecisionTreeClassifier(max_depth=8)
#knn = KNeighborsClassifier(n_neighbors=2)
#svm = SVC(kernel='linear',C=0.025)

In [14]:
ada_oof_train,ada_oof_test = get_out_fold(ada,X_train.values,y_train.values,X_test.values)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').